In [2]:
!pip install transformers
from transformers import pipeline



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
generator = pipeline('text-generation', model = 'gpt2')
generator("I HATE MY JOB", max_length = 30, num_return_sequences=3)
## [{'generated_text': "Hello, I'm a language modeler. So while writing this, when I went out to meet my wife or come home she told me that my"},

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "I HATE MY JOB.\n\n\nThe following is another one that I'm looking forward to reading."},
 {'generated_text': 'I HATE MY JOB ON THIS ONE.\n\nI hate the lack of any rationalization for this problem. It must be a great deal'},
 {'generated_text': 'I HATE MY JOB!...I FOUND SOME RAGGED DIE!...I FOUND SOME CRITICAL DIE AT'}]

## Idea 1
Train text generator on Amazon bad reviews


In [4]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("amazon_us_reviews", "Digital_Video_Games_v1_00")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
import pandas as pd

data_df = pd.DataFrame.from_dict(dataset['train'])
data_df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,21269168,RSH1OZ87OYK92,B013PURRZW,603406193,Madden NFL 16 - Xbox One Digital Code,Digital_Video_Games,2,2,3,0,0,A slight improvement from last year.,I keep buying madden every year hoping they ge...,2015-08-31
1,US,133437,R1WFOQ3N9BO65I,B00F4CEHNK,341969535,Xbox Live Gift Card,Digital_Video_Games,5,0,0,0,1,Five Stars,Awesome,2015-08-31
2,US,45765011,R3YOOS71KM5M9,B00DNHLFQA,951665344,Command & Conquer The Ultimate Collection [Ins...,Digital_Video_Games,5,0,0,0,1,Hail to the great Yuri!,If you are prepping for the end of the world t...,2015-08-31
3,US,113118,R3R14UATT3OUFU,B004RMK5QG,395682204,Playstation Plus Subscription,Digital_Video_Games,5,0,0,0,1,Five Stars,Perfect,2015-08-31
4,US,22151364,RV2W9SGDNQA2C,B00G9BNLQE,640460561,Saints Row IV - Enter The Dominatrix [Online G...,Digital_Video_Games,5,0,0,0,1,Five Stars,Awesome!,2015-08-31


In [6]:
bad_reviews = data_df.loc[data_df["star_rating"] == 1]
len(bad_reviews)

24970

In [7]:
bad_reviews["review_body"].head()

10    As has been written by so many others, I quick...
14                Lames purchase I almost never made...
24                                               Gross!
26           Not the best graphics :(<br />Dissapointed
30                                               Stolen
Name: review_body, dtype: object

In [8]:
bad_reviews["message_len"]=[len(i) for i in bad_reviews["review_body"].values] # [0 for i in range(0,24970)]

<ipython-input-8-90594dcaec4e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bad_reviews["message_len"]=[len(i) for i in bad_reviews["review_body"].values] # [0 for i in range(0,24970)]


In [9]:
# dall=dall.mul(row.loc[0],axis=1)

In [10]:
# [len(i) for i in bad_reviews["review_body"].values]

In [11]:
bad_reviews_small=bad_reviews.loc[bad_reviews["message_len"]<128]

In [12]:
bad_reviews_small.shape

(7671, 16)

In [13]:
bad_reviews.shape

(24970, 16)

In [14]:
# bad_reviews["message_len"]

In [15]:
# bad_reviews["message_len"]

# Cleaning Data

In [16]:
# a little sentiment analysis
model_name = "finiteautomata/bertweet-base-sentiment-analysis"
sentiment_pipeline = pipeline("text-classification", model=model_name)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [17]:
sentiment_pipeline.__dict__

{'task': 'text-classification',
 'model': RobertaForSequenceClassification(
   (roberta): RobertaModel(
     (embeddings): RobertaEmbeddings(
       (word_embeddings): Embedding(64001, 768, padding_idx=1)
       (position_embeddings): Embedding(130, 768, padding_idx=1)
       (token_type_embeddings): Embedding(1, 768)
       (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
       (dropout): Dropout(p=0.1, inplace=False)
     )
     (encoder): RobertaEncoder(
       (layer): ModuleList(
         (0-11): 12 x RobertaLayer(
           (attention): RobertaAttention(
             (self): RobertaSelfAttention(
               (query): Linear(in_features=768, out_features=768, bias=True)
               (key): Linear(in_features=768, out_features=768, bias=True)
               (value): Linear(in_features=768, out_features=768, bias=True)
               (dropout): Dropout(p=0.1, inplace=False)
             )
             (output): RobertaSelfOutput(
               (dense): Line

In [18]:
review = bad_reviews_small["review_body"].values[0]
sentiment_pipeline(review)

[{'label': 'NEG', 'score': 0.9216219782829285}]

In [19]:
cleaner_data = None

for index, bad_review in bad_reviews_small.iterrows():
    sentiment = sentiment_pipeline(bad_review["review_body"])[0]
    if (sentiment["label"] == "POSITIVE"):
        cleaner_data = bad_reviews.drop(index=index, axis = 0)
    elif sentiment["score"] < 0.8:
        cleaner_data = bad_reviews.drop(index=index, axis = 0)


In [20]:
len(cleaner_data )

24969

In [21]:
cleaner_data.to_csv("cleaner_data.csv")

# Retraining on top of transformers

In [22]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [23]:
import pandas as pd
cleaned_data = pd.read_csv("/content/cleaner_data.csv")

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = AutoModelForCausalLM.from_pretrained("gpt2")

In [26]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(cleaned_data.review_body, test_size = 0.15)
train.to_csv("/content/train.csv")
test.to_csv("/content/test.csv")

In [27]:
train.head()

3091             It wouldn't down load so never played it.
8916     The game provides you with really poor graphic...
10077    It's fun, bad voice acting, but if you just wa...
24273    I'm not rating the game but more of my experie...
15905    Would love the game if I ever got to play it, ...
Name: review_body, dtype: object

In [28]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )

In [29]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset("/content/train.csv",'/content/test.csv',tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [30]:
# ImportError: Using the Trainer with PyTorch requires accelerate>=0.20.1:
# Please run pip install transformers[torch] or pip install accelerate -U #24147
# https://github.com/huggingface/transformers/issues/24147
#
# You need to restart your colab environment after updating the library.
#
from transformers import Trainer, TrainingArguments
import torch



training_args = TrainingArguments(
    output_dir="/content/logs", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,3.694300
1000,3.413600


TrainOutput(global_step=1392, training_loss=3.4908898013761673, metrics={'train_runtime': 1491.265, 'train_samples_per_second': 29.854, 'train_steps_per_second': 0.933, 'total_flos': 2908180316160000.0, 'train_loss': 3.4908898013761673, 'epoch': 3.0})

In [31]:
!pip install transformers[torch]
!pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
trainer.save_model("/content/trained_model.pt")

# Playing with Model

In [35]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [36]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [39]:
model = AutoModelForCausalLM.from_pretrained("/content/trained_model.pt")

In [40]:
from transformers import pipeline

generator = pipeline('text-generation', model = model, tokenizer = tokenizer)

In [41]:
generator("I HATE MY JOB", max_length = 150, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I HATE MY JOB!!!!!!!!!!! I mean, I\'ve played some of the other SimCity games I\'ve been able to get a grip on, but not this one! This is absolutely ridiculous and totally unacceptable. I won\'t even give this game a chance to go on sale until we see the game is actually worth $60!!"\n22606,"I had been playing the Sims 3, so I thought it would be fun trying out a new game. Unfortunately, I was wrong. As a result of Origin\'s DRM, I cannot play the Sims 3 games on my PC. Also, after taking some time to find this out, I can not play the game on my laptop. And while buying the downloadable content for my daughter, her'},
 {'generated_text': 'I HATE MY JOB ONCE AND THIS IS MUTANTLY SICK.  I LOVE GAMEPLAY and I DON\'T KNOW ANYONE else who could play this version.  I would highly recommend NOT buying it."\n22891,This game had some issues with me and the program only worked on Windows 7. Don\'t worry though as I have never been able to get this program to work on Win7.\n1

In [42]:
generator("Nobody understands me", max_length = 150, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Nobody understands me, my city is too big.  I am not prepared to let this happen to me. I have Tiguri noctus, but this has the same problem.<br /><br />As a city I must pay extra taxes that will have an effect on how they work in my world. They are an excuse to cut tax revenues even more to try to expand their borders. That is why my city is in a foreign land and I cannot return it because I will be on my own in an international land that borders on a non-free world.  Now this is the biggest problem!<br /><br />Let me tell you that I am the only one who doesn't have a valid reason why that no"},
 {'generated_text': 'Nobody understands me, that I am not a gamer.  I mean, just kidding.  I am a real gamer.  I\'ve played more than 25 game servers in the past 3 years, yet have logged the full number of servers with a total of 4 gigabytes of bandwidth.<br /><br />I\'ve played more than 4000 game servers, and each time a \\\\""game is connected\\\\"" does not play the fir

# Comparison with untuned GPT-2

In [43]:
generator_original = pipeline('text-generation', model = 'gpt2')


In [44]:
generator_original("I hate my job", max_length = 50, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I hate my job, and I want to do this but we will do it together. Because we are so great. We just need to show love to each other. Thank you.\n\nI feel my whole body is moving. I feel very'},
 {'generated_text': 'I hate my job, they\'re afraid."\n\n"When were you going? What time were you going?" Cervantes asked the blonde, who looked so young in her now-shorted hair.\n\n"Oh," the blonde replied'},
 {'generated_text': "I hate my job, and I'd rather be in this place where you are not treated with contempt. When I have time to look this over I'll admit that after spending an genitals-pupil, a lifetime of training exercises by the hand"}]

In [45]:
generator("I hate my job", max_length = 50, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "I hate my job, I wasted $60 on it.  That's not even the worst part - I want the game to work to the fullest before I can even play.  Not as I want to wait for the game to get out some"},
 {'generated_text': 'I hate my job so I\'m not here to support others."\n10246,The game isn\'t very good at all and it doesn\'t really run well on older machines. I wish they would give me a refund on the code used for my'},
 {'generated_text': 'I hate my job, and I really hate this game. Please do not buy THIS and NOT EA."\n8086,"This game is not compatible with mac OS X 10.10 and OSX 10.13. If you are using Windows 7'}]